# Introduction
A perceptron model, sometimes also referred as a single neuron neural network or a linear learning model. It basically performs a weighed sum of the input with weights, adds a bias and then passes the value through an _activation function_. 
In short, the steps to get an output are
1. Perform weighed sum and add bias
2. Pass the obtained value through a function

We'll use the famous _sigmoid_ activation function here, where

$$sigmoid(x) = \frac{1}{1 + e^{-x}}$$

Therefore, to summarize. We'll do the following in the right order to forward propagate. (This is what a perceptron does)

$$Z = W*X + b$$
$$A = sigmoid(Z)$$

Here, *X* is an input vector (column type), *W* is the weights vector (row type) and *b* is the bias.

### _Hand Gesture Recognition_
Here are the different hand gestures for this purpose. We have another gesture (gesture number 0) which shows no fingers (just a closed fist) and is not shown here
![Hand Gestures](Hand-Gestures-count.jpg "Hand Gestures in the database")

For now we will consider the gesture number **5**, and we'll train a perceptron to recognize that gesture with a descent accuracy. In total we have 6 gestures

Let's start by importing the files we need

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

## Importing data we need
We have approximately 231 training examples from each gesture type
This data is in form of [235, 190] images which are shrunk to [47, 38] images using image interpolation, this is done because otherwise there will be too many parameters for the perceptron to deal with. The data acquisition process was as following
1. Get black and white binary images of shape [235, 190]
2. Use interpolation and shrink them to shape [47, 38]
3. Flatten every image into a column vector and stack all those column vectors together

There are many techniques to generate more data from the data that you already have. These techniques are called _data augmentation_.

Fortunately, all this work has already been done for us and the variables are stored in binary files. We only have to load them into memory. Additionally the binary images have two values, 0 and 255 (because the were 8 bit numbers). We will scale them down to the range 0 to 1. This makes the inputs smaller and all of them will be in range. This technique is called _data normalization_ and increases the efficiency of the learning algorithm.

In [ ]:
# Function to load the data variables from memory into the program
def load_data_variables(output_number = 5, normalize_inputs = True, view_sample_data = True):
    # Load 47 x 38 images (input data)
    X = np.load('Data/X.npy')
    # Load one hot encoded output data
    Y_one_hot = np.load('Data/Y_one_hot_encoded.npy')
    # Get the 5th row (gesture number 5)
    Y = np.array(Y_one_hot[output_number,:])
    # Assert it to be a row vector
    Y = Y.reshape((1,-1))
    # Viewing data sample
    if view_sample_data:
        print("DATA DEBUG : Viewing 100 examples")
        view_100_examples(X)
    # Normalize data
    if normalize_inputs:
        X = X/255
    return X,Y

Let's now write some code to display a 10 x 10 image sample using matplotlib. Note that _Greys_ will display inverted binary images (swap 0 and 1), this is done just to suit the background

In [ ]:
# Show input samples
def view_100_examples(X):
    # View 100 random 47 x 38 images from the dataset
    for i in range(1, 101):
        rno = np.random.randint(X.shape[1])
        img = X[:,rno].reshape(47, 38)
        plt.subplot(10, 10, i)
        plt.imshow(img, cmap='Greys')
        plt.axis('off')
    plt.suptitle("Random Images")
    plt.show()

Let's now finally load our data

In [ ]:
print("Loading variables")
X, Y = load_data_variables()
print("DATA INFO : Variables loaded, input shape is {x_shape}, output shape is {y_shape}".format(
    x_shape=X.shape, y_shape= Y.shape
))

Expected output : 

Feature | Shape
-------|------
input shape | (1786, 2772)
output shape | (1, 2772)

This is because we have 231 training examples of each gesture and we have 6 gestures. Now that gets 231 * 6 = 1386 training examples. But since we performed data augmentation we have 2 images for one input (flipped versions). Thus we have 1386 * 2 = 2772 training examples. Input has 47 * 38 = 1786 fields (rows) and output has just one row. So numbers match. Let's move forward with shuffling this data. This is a very good practice to breat a biased distribution from a source. This always let's us "learn" better from our data. We will just shuffle the columns in our data.

In [ ]:
# Shuffle data
def shuffle_data(X, Y, iternum = 1):
    print("DATA DEBUG : Shuffling data for {num_iter} iterations".format(num_iter=iternum))
    # Make a dummy matrix
    sh_mat = X
    # Row stack the output matrix
    sh_mat = np.row_stack((sh_mat, Y))
    sh_mat = sh_mat.T   # Because the np.random.shuffle shuffle's rows
    for i in range(iternum):
        np.random.shuffle(sh_mat)
    sh_mat = sh_mat.T  # Get back the data into column form
    X = sh_mat[0:X.shape[0], :]                            # Shuffled input data
    Y = sh_mat[X.shape[0]:X.shape[0] + Y.shape[0], :]      # Shuffled output data
    return np.array(X), np.array(Y)

In [ ]:
X, Y = shuffle_data(X, Y)

### Split the data
We now need to split the data into three sets (three buctets). Namely, split the data into **train**, **dev** and **test** sets. <br>
Why do that ? <br>
Because it rarely happens that we try out only one type of model and that is the best choice. We take many models into consideration. For instance, I might want to know that how does a different activation function affect the final accuracy. So we have one metric to decide the performance of our ML model. We call this the _performance metric_. So we train our models on the train set, compare different models using the dev set. The test set is more like a final run performance. Once we've chosen the best model, we want to know how our model will perform on real world data. For this, we have to check performance on the data that the model hasn't seen yet, hence the test set. Formally, 

Set | Purpose
----|----
Train | To train the model
Dev | To compare different models
Test | To test the final performance of our model

Sometimes, the dev set is also called the "cross validation" set.<br>
Now how do we choose the distribution ratio ?<br>
Well, we aim to put most of the data into the training bucket, and a few amount in dev and test sets. Another important thing is that we need the dev and test sets to be from nearly the same probability distribution. This is so that we can choose the most robust model and also gain some knowledge about the changes we have to make to the model. The performance metric gives us insight into our model, if it's overfitting our data (causing too much variance) or underfitting our data, causing too much bias. We'll deal with _bias, variance tradeoff_ later. For now, let's go with 2500 training, 136 dev and test set examples

In [ ]:
def generate_train_dev_test(X, Y, sizes=None, shuffle=False):
    # Split given data into train, dev and test sets
    if sizes is None:
        # Default sizes
        size_train = 2500
        size_dev = 136
        size_test = 136
    else:
        (size_train, size_dev, size_test) = sizes
        assert (size_train + size_test + size_dev <= X.shape[1]), "Size mismatch error {v1}>{v2}".format(
            v1=size_train + size_test + size_dev, v2=X.shape[1]
        )
    if shuffle:
        X, Y = shuffle_data(X, Y)
    x_train = np.array(X[:,:size_train])
    print("DATA DEBUG : x_train is of shape {x_train_shape}".format(
        x_train_shape=x_train.shape
    ))
    y_train = np.array(Y[:,:size_train])
    print("DATA DEBUG : y_train is of shape {y_train_shape}".format(
        y_train_shape=y_train.shape
    ))
    x_dev = np.array(X[:,size_train: size_train + size_dev])
    print("DATA DEBUG : x_dev is of shape {x_dev_shape}".format(
        x_dev_shape=x_dev.shape
    ))
    y_dev = np.array(Y[:, size_train: size_train + size_dev])
    print("DATA DEBUG : y_dev is of shape {y_dev_shape}".format(
        y_dev_shape=y_dev.shape
    ))
    x_test = np.array(X[:, size_train + size_dev: size_train + size_dev + size_test])
    print("DATA DEBUG : x_test is of shape {x_test_shape}".format(
        x_test_shape=x_test.shape
    ))
    y_test = np.array(Y[:, size_train + size_dev: size_train + size_dev + size_test])
    print("DATA DEBUG : y_test is of shape {y_test_shape}".format(
        y_test_shape=y_test.shape
    ))
    rdict = {
        "train": (x_train, y_train),
        "dev": (x_dev, y_dev),
        "test": (x_test, y_test)
    }
    return rdict

In [ ]:
data_sets = generate_train_dev_test(X, Y)
(X_train, Y_train) = data_sets["train"]
(X_dev, Y_dev) = data_sets["dev"]
(X_test, Y_test) = data_sets["test"]

Expected output

Set | Shape
------|------
X-train | (1786, 2500)
Y-train | (1, 2500)
X-dev | (1786, 136)
Y-dev | (1, 136)
X-test | (1786, 136)
Y-test | (1, 136)

## Main model
Now, we start the main model code
Remember, our model does a weighed sum and then passes it through sigmoid function. This is sometimes also written as **Linear -> Sigmoid**. The maths on this is shown below
![Perceptron math](IMG_0233.JPG "Mathematical derivation")

The two main parts are in black. Now they are for row major data, we're dealing with column major (so change axis of summation). We'll discuss this when the time to program the model comes

For now, let's initialize parameters "W" and "b". Remember that **W** is (1, 1786) and **b** is (1, 1).
The main question is, what do we initialize the parameters to ?<br>
Well, if we initialize all the weights to zero and the bias to zero, then we will predict zero as output. In case of perceptron, this might work sometimes, but this is a bad practice, because when we have a bigger model, initializing all weights to zeros will end up in all weights learning the same thing. This dampens the learning of our model and significantly reduces the performance. For now, we initialize the weights with random numbers. The method I used is called _He initialization_. For the bias, it doesn't matter if it's initialized to 0 or not, it's just one value.

The code behind this is
```python
params["W"] = np.random.rand(1, n) * (2/np.sqrt(n-1))
params["b"] = np.zeros((1,1))
```

In [ ]:
def initialize_parameters(X, Y, seedval=2):
    # Initialize parameters
    n = X.shape[0]      # Number of fields
    def initialize_zeros():
        W = np.zeros((1, n))
        b = np.zeros((1, 1))
        rdict = {
            "W": W,
            "b": b
        }
        return rdict
    params = initialize_zeros()
    np.random.seed(seedval)
    W = params['W']
    b = params['b']
    # Initialize weights
    W = np.random.rand(*W.shape) * (2/np.sqrt(n - 1))
    # Initialize bias
    b = np.zeros(b.shape)
    print("DATA DEBUG : Weights of shape {w_shape}, sample is {w_sample}".format(
        w_shape=W.shape, w_sample=W
    ))
    print("DATA DEBUG : Bias of shape {b_shape}, sample is {b_sample}".format(
        b_shape=b.shape, b_sample=b
    ))
    params = {
        "W": W,
        "b":b
    }
    return params

In [ ]:
params = initialize_parameters(X_train, Y_train)

Expected output

Parameter | Shape | Initial value
------|--------|------
"W"| (1, 1786) | [[0.02063917 0.0012273 ... 0.02258119 0.00640937]]
"b"| (1, 1) | [[0.]]

Now that we have initialized the parameters, let us declare the sigmoid function and it's derivative.

In [ ]:
# Sigmoid functions
def sigmoid(x):
    # Return sigmoid of an input
    return np.exp(x)/(1 + np.exp(x))
# Sigmoid derivative
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [ ]:
print("TEST : Sigmoid = {sgm_res}".format(sgm_res=sigmoid(0)))
print("TEST : Sigmoid Derivative = {sd_res}".format(
    sd_res=sigmoid_derivative(0.5)))

Expected output

Function | Result
----|----
Sigmoid | 0.5
Sigmoid Derivative | 0.235

Now, we must implement the forward propagation step. Remember that the parameter **W** is the weight of every field, so the weighed sum with bias of the entire input space can be calculated by $Z = W * X + b$. Here, matrice multiplication is used (between _W_ and _X_).
We then obtain the output through the sigmoid function
$$A = sigmoid(Z)$$

In [ ]:
# Forward propagation function
def forward_propagate(params, X):
    # Forward propagation step
    W = params["W"]
    b = params["b"]
    Z = np.matmul(W, X) + b
    A = sigmoid(Z)
    return A

In [ ]:
# To generate the test examples for the forward propagation step
def generate_forward_propagation_test():
    test_x = np.arange(6).reshape((3, 2))
    test_W = np.array([0.2, 0.5, 0.6]).reshape((1, -1))
    test_b = np.array([3])
    rdict = {
        "W":test_W,
        "b":test_b
    }
    return rdict, test_x

test_params, test_x = generate_forward_propagation_test()
print("TEST : Forward Propagation test = {fp_res}".format(
    fp_res=forward_propagate(test_params, test_x)
))

Expected output

 Test | Result
----|----
Forward Propagation| [[0.9983412, 0.99954738]]

### Cost function
This is the function through which we must optimize. This is often denoted by **J**. 
We use the logistic regression cost function for classification based problems, also known as the _cross entropy cost function_.
The cost function performed on just one training example is also called the loss function. This function actually takes in two values, the predictions *A* (or also called $\hat{Y}$) and the actual value *Y*.
The cross entropy loss is given by

$$Loss(A,Y) = -(Y \times ln(A) + (1-Y) \times ln(1-A))$$
<center>**OR**</center>
$$Loss(\hat{Y},Y) = -(Y \times ln(\hat{Y}) + (1-Y) \times ln(1-\hat{Y}))$$

The cost is just the average value of loss over the entire training set. Hence the cross entropy cost funciton is 
$$\Rightarrow Cross(A,Y) = \frac{1}{m} \sum_{k = 1}^{m} Loss(A^{k}, Y^{k})$$

But, we will implement a different cost function in code. This loss function is simply given by the distance between the two vectors A and Y squared multiplied by `0.5`. Mathematically

$$Cost(A, Y) = \frac{||A - Y||^{2}}{2\, m}$$

This is being done because we've calculated optimization formula through hand in the coming cells, so we'll only be needing this for verification purposes and it's easier to interpret results through this type of cost funciton

In [ ]:
# Cost function
def compute_cost(params, X, Y):
    # Compute the cost
    A = forward_propagate(params, X)
    L = np.square(A - Y)
    return np.average(L)/2

### Gradient Descent
Every weight and bias is adjusted through an optimizing function. The simplest optimizing function is _gradient descent_. This algorithm takes a small step towards the minima.
<center>**Gradient Descent algorithm**</center>
![Gradient Descent algorithm](gradient-descent.png "Gradient Descent")

$$W_{k} := W_{k} - \alpha \times  \frac{\partial J}{\partial W_{k}}$$
$$b := b - \alpha \times  \frac{\partial J}{\partial b}$$

(The := means compute whatever is on the right and assign it to the left variable)<br>
If we differentiate the _cross entropy cost function_ and apply the _chain rule of differentiation_ we get the following results

$$W_{k} := W_{k} - \alpha \times [\frac{1}{m} \sum_{i = 1}^{m} (A^{i} - Y^{i}) \times X_{k}^{i}]$$
$$b := b - \alpha \times [\frac{1}{m} \sum_{i = 1}^{m} (A^{i} - Y^{i})]$$

In case you're wondering, $X_{k}^{i}$ stands for the k<sup>th</sup> feature of i<sup>th</sup> training example. In our column major form, it's the k<sup>th</sup> row and i<sup>th</sup> column of **X**. **W**<sub>k</sub> stands for the weight of the k<sup>th</sup> feature.

A vectorized implementation would we great since they're optimized. In python, we can carry this out through the following
```python
W = W - (learning_rate/m) * np.sum(X * (A - Y), axis=1, keepdims=True).T
b = b - (learning_rate/m) * np.sum((A - Y), axis=1, keepdims=True).T
```
If you pay closer attention, it's not so difficult to understand. X * (A - Y) computes (A - Y) and then performs row wise multiplication with X, then we add all the columns. The transpose (.T) is just to convert the delta (gradient) from a column vector to a row vector, to match the dimensions of W. Same story goes with b

This tells us that there's a way simpler implementation of the cost function. One which is more efficient to calculate and will do the exact same thing if we tune the optimization code as well.

In [ ]:
# Optimization code
def train_parameters_gradient_descent(params, X, Y, num_iter = 100, learning_rate = 0.01, print_cost = True):
    # Get shape
    (n, m) = X.shape
    assert (m == Y.shape[1]), "ERROR : Dataset mismatch"
    # Store the history of costs
    cost_history = {
        "training_costs_x" : [],   # i values to plot w.r.t
        "training_costs": [],      # cost during training
        "test_costs_x" : [],       # i values to plot w.r.t
        "test_costs": []           # cost over the test set during training
    }
    # Iterate over the entire batch of data
    for i in range(num_iter):
        # Forward propagate to calculate prediction
        A = forward_propagate(params, X)
        # Get weights and biases
        W = params["W"]
        b = params["b"]
        # Gradient Descent optimization algorithm
        W = W - (learning_rate/m) * np.sum(X * (A - Y), axis=1, keepdims=True).T
        b = b - (learning_rate/m) * np.sum((A - Y), axis=1, keepdims=True).T
        # Update parameters
        params["W"] = W
        params["b"] = b
        current_cost = compute_cost(params, X, Y)
        # Show output after some number of iterations to report training
        if print_cost and i % 10 == 0:
            print("TRAINING INFO : Iteration {iter}, cost = {cst}".format(
                iter= i, cst=current_cost))
            print("TRAINING DEBUG : W {wsh}, X {xsh}, b {bsh}, A {ash}, Y {ysh}".format(
                    wsh=params["W"].shape, xsh=X.shape, bsh=params["b"].shape,
                    ash=A.shape, ysh= Y.shape))
            cost_test = compute_cost(params, X_dev, Y_dev)
            cost_history["test_costs"].append(cost_test)
            cost_history["test_costs_x"].append(i + 1)
        cost_history["training_costs"].append(current_cost)
        cost_history["training_costs_x"].append(i + 1)
    return cost_history, params

Now we've implemented everything required to train, let us train our perceptron and get the results.<br>
**WARNING** : Run the below cell only once to get the expected output, if you want to run it again, please re initialize the weights

In [ ]:
# Already initialised stuff (uncomment if you feel like)
# params = initialize_parameters(X_train, Y_train)
# Actual training
number_iterations = 100
c_hist, params = train_parameters_gradient_descent(params, X_train, Y_train, num_iter=number_iterations)
plt.plot(c_hist["training_costs_x"], c_hist["training_costs"], 'b-', c_hist["test_costs_x"], c_hist["test_costs"], 'g-')
plt.title('Cost History')
plt.legend(['Training','Dev'])
plt.show()
print("TRAINING DEBUG : Training done.")

Expected output

Iteration | Cost
---|---
0 | 0.4159
10 | 0.1023
$\vdots$ | $\vdots$
90 | 0.07683

The graph in the end shows us an important result. It has captured the progress of training the model. If you don't see a descent in the blue line (if it's raising), it indicates that there has been an error. We have very unprofessionally used the dev set to check the perforce while training. Usually there's a fourth set for this, it's most of the times called the 'Training_dev set'. Training and evaluation is a very important metric and is used by the best of libraries which do such kind of tasks. It enables them to gain insight into the training process.

## Error analysis

Now that we've trained the model, it's time we check out it's performance on the test set as well. We'll use a function for this

In [ ]:
# Compute the error on the test set
def error_test_set(params, test_x, test_y, view_mismatches=True):
    # Number of test samples
    (_, m) = test_y.shape
    # Get predictions through forward propagation
    A = forward_propagate(params, test_x)
    print("TEST DEBUG : W {wsh}, X {xsh}, b {bsh}, Y {ysh}".format(
        wsh=params["W"].shape, xsh= X.shape, bsh= params["b"].shape, ysh= test_y.shape
    ))
    # Threshold places where the confidence level is > 50 %
    B = np.zeros_like(A)
    B[A > 0.5] = 1
    # Compute the mismatches and error
    D = np.square(test_y - B)
    D = np.array(D)
    # Wherever D is 1, it's a mismatch
    mismatches = D.nonzero()[1]
    print("TEST DEBUG : {num_mis} mismatches (out of {num}) found".format(
        num_mis= mismatches.shape[0], num=test_y.shape[1]))
    if view_mismatches == True:
        # View mismatches 
        print("TEST DEBUG : Viewing mismatches")
        for i in range(len(mismatches)):
            plt.imshow(test_x[:,mismatches[i]].reshape((47,38)), cmap='Greys')
            plt.title("A = {a}, Y = {y}".format(a=round(A[0, mismatches[i]], 2), y=test_y[0, mismatches[i]]))
            plt.show()
    error = np.sum(D)/m
    return error * 100

Now let's calculate the error

In [ ]:
error = error_test_set(params, X_test, Y_test, view_mismatches= False)
print("TEST DEBUG : Error is {err}%".format(err=error))
print("Accuracy is {acc}%".format(acc=100-error))

Expected output 

Field | Value
--|--
Number of mismatches | $\approx$ 22 out of 136
Error percentage | $\approx$ 16.17647 %
Accuracy | $\approx$ 83.82 %

We have finally made a linear classifier that can detect a 'Five finger' gesture which has 83.8 % accuracy. This is pretty awesome

In [ ]:
error = error_test_set(params, X_test, Y_test, view_mismatches= True)
print("TEST DEBUG : Error is {err}%".format(err=error))
print("Accuracy is {acc}%".format(acc=100-error))

What we did above gives us even more detailed insight into our learned parameters. It shows what it's exactly predicting as 'Five finger gesture' and what it's not predicting as a 'Five finger gesture'. We can see that if we bring any kind of swirl or change the positions from the standard, we get an invalid result. This is because the perceptron cannot get that level of complexity. We cannot encode that high amount of complexity in a single neuron. However, this performance isn't very bad.

Feel free to play with more models below

In [ ]:
# Load variables and divide into test, train and dev sets
X, Y = load_data_variables(output_number=3)
data_sets = generate_train_dev_test(X, Y, shuffle=True)
(X_train, Y_train) = data_sets["train"]
(X_dev, Y_dev) = data_sets["dev"]
(X_test, Y_test) = data_sets["test"]
# Initialize random parameters
params = initialize_parameters(X_train, Y_train)
# Train
number_iterations = 100
c_hist, params = train_parameters_gradient_descent(params, X_train, Y_train, num_iter=number_iterations)
plt.plot(c_hist["training_costs_x"], c_hist["training_costs"], 'b-', c_hist["test_costs_x"], c_hist["test_costs"], 'g-')
plt.title('Cost History')
plt.legend(['Training','Dev'])
plt.show()
print("TRAINING DEBUG : Training done.")
# Error analysis
error = error_test_set(params, X_test, Y_test, view_mismatches= True)
print("TEST DEBUG : Error is {err}%".format(err=error))
print("Accuracy is {acc}%".format(acc=100-error))

The best results I got are

Gesture | Accuracy
--|--
5 | 83.823 %
4 | 81.617 %
3 | 77.941 %
2 | 83.088 %
1 | 83.088 %
0 | 86.764 %

Well, this clearly isn't the best we can do. There are multiple directions to head from here

- Get edges from mask and use that as training data
- Build a more sophisticated model (like a DNN, ANN or a CNN)